# Minimal three node network

Here, we are going to create a network with three nodes, three lines and one generator. We then solve the non-linear power flow using a Newton-Raphson.  

In [1]:
import pypsa
import numpy as np

In [2]:
network = pypsa.Network()

Add three buses

In [3]:
n_buses = 5

for i in range(n_buses):
    network.add("Bus", "Bus {}".format(i), v_nom=220.0)

network.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
Bus 0,220.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
Bus 1,220.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
Bus 2,220.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
Bus 3,220.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
Bus 4,220.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,


加入三根导线

In [4]:
# A --> 1
network.add(
    "Line",
    "Line 1",
    bus0 = "Bus 0",
    bus1 = "Bus 1",
    r = 10.63,
    x = 14.75,
)
# A --> S
network.add(
    "Line",
    "Line 2",
    bus0 = "Bus 0",
    bus1 = "Bus 2",
    r = 5.2,
    x = 16.32,
)
# A --> 2
network.add(
    "Line",
    "Line 3",
    bus0 = "Bus 0",
    bus1 = "Bus 3",
    r = 11.22,
    x = 19.47,
)
# A --> 3
network.add(
    "Line",
    "Line 4",
    bus0 = "Bus 0",
    bus1 = "Bus 4",
    r = 10.16,
    x = 17.48,
)

Add three lines in a ring

In [5]:
'''
for i in range(n_buses):
    network.add(
        "Line",
        "My line {}".format(i),
        bus0="My bus {}".format(i),
        bus1="My bus {}".format((i + 1) % n_buses),
        x=0.1,
        r=0.01,
    )
'''
network.lines

attribute,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,n_mod
Line,,,,,,,,,,,,,,,,,,,,,
Line 1,Bus 0,Bus 1,,14.75,10.63,0.0,0.0,0.0,0.0,False,...,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
Line 2,Bus 0,Bus 2,,16.32,5.20,0.0,0.0,0.0,0.0,False,...,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
Line 3,Bus 0,Bus 3,,19.47,11.22,0.0,0.0,0.0,0.0,False,...,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
Line 4,Bus 0,Bus 4,,17.48,10.16,0.0,0.0,0.0,0.0,False,...,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


Add a generator at bus 0

In [6]:
'''
network.add("Generator", "My gen", bus="My bus 0", p_set=300, control="PQ")

network.generators
'''
network.add("Generator", "发电厂", bus = "Bus 0", p_set = 600, q_set= 261.53, control="PQ")

In [7]:
network.generators.p_set

Generator
发电厂    600.0
Name: p_set, dtype: float64

Add a load at bus 1

In [8]:
'''
network.add("Load", "My load", bus="My bus 1", p_set=100)
'''
network.add("Load", "Load A1", bus="Bus 1", p_set=89, q_set=37.81)
network.add("Load", "Load AS", bus="Bus 2", p_set=229, q_set=103.5)
network.add("Load", "Load A2", bus="Bus 3", p_set=150, q_set=63.89)
network.add("Load", "Load A3", bus="Bus 4", p_set=132, q_set=56.23)
network.loads

attribute,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
Load A1,Bus 1,,,89.0,37.81,-1.0
Load AS,Bus 2,,,229.0,103.50,-1.0
Load A2,Bus 3,,,150.0,63.89,-1.0
Load A3,Bus 4,,,132.0,56.23,-1.0


In [9]:
network.loads.p_set

Load
Load A1     89.0
Load AS    229.0
Load A2    150.0
Load A3    132.0
Name: p_set, dtype: float64

Fix the reactive power of the load

In [10]:
# network.loads.q_set = 100.0

Do a Newton-Raphson power flow

In [11]:
network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.217447 seconds


{'n_iter': SubNetwork  0
 snapshot     
 now         4,
 'error': SubNetwork             0
 snapshot                
 now         4.547474e-13,
 'converged': SubNetwork     0
 snapshot        
 now         True}

Alright, it converged! Now, what is the active power flow on the lines?

In [12]:
network.lines_t.p0

,Line 1,Line 2,Line 3,Line 4
snapshot,,,,
now,91.193016,236.781711,157.06279,136.801601


...and what are the voltage angles on the buses?

In [13]:
network.buses_t.v_ang * 180 / np.pi

Bus,Bus 0,Bus 1,Bus 2,Bus 3,Bus 4
snapshot,,,,,
now,0.0,-1.114292,-4.059069,-2.79392,-2.166862


...and their mangitudes?

In [14]:
network.buses_t.v_mag_pu

Bus,Bus 0,Bus 1,Bus 2,Bus 3,Bus 4
snapshot,,,,,
now,1.0,0.967704,0.933769,0.934069,0.94867


In [18]:
network.lines_t.q1

,Line 1,Line 2,Line 3,Line 4
snapshot,,,,
now,-37.81,-103.5,-63.89,-56.23


In [19]:
network.lines_t.p1

,Line 1,Line 2,Line 3,Line 4
snapshot,,,,
now,-89.0,-229.0,-150.0,-132.0


In [21]:
network.lines_t.p0

,Line 1,Line 2,Line 3,Line 4
snapshot,,,,
now,91.193016,236.781711,157.06279,136.801601


In [20]:
network.lines_t.q0

,Line 1,Line 2,Line 3,Line 4
snapshot,,,,
now,40.85299,127.9226,76.146017,64.491023
